In [1]:
# This analysis was already done, but now we are using a more detailed dataset with more points

In [27]:
import pandas as pd
import json
import requests
from urllib.request import urlopen

In [28]:
key = '05fcc51d368d71bce9d7627d3097d978'
projectId = '82d5e815'

In [29]:
ages = pd.read_csv(r'../data/original/building.csv')
ages.head()

,BIN,CNSTRCT_YR,NAME,LSTMODDATE,LSTSTATYPE,the_geom,DOITT_ID,HEIGHTROOF,FEAT_CODE,GROUNDELEV,SHAPE_AREA,SHAPE_LEN,BASE_BBL,MPLUTO_BBL,GEOMSOURCE
0,3394646,2009.0,NaN,08/22/2017 12:00:00 AM +0000,Constructed,MULTIPOLYGON (((-73.87129515296562 40.65717370...,1212853,21.608508,2100.0,18.0,854.662433,125.079796,3.044521e+09,3.044521e+09,Photogramm
1,4548330,1930.0,NaN,08/17/2017 12:00:00 AM +0000,Constructed,MULTIPOLYGON (((-73.87670970144625 40.71425234...,1226227,10.360000,5110.0,122.0,217.594243,60.225858,4.030640e+09,4.030640e+09,Photogramm
2,4460479,1960.0,NaN,08/22/2017 12:00:00 AM +0000,Constructed,MULTIPOLYGON (((-73.85195485799383 40.66235471...,581946,29.811570,2100.0,10.0,946.427476,123.141941,4.139430e+09,4.139430e+09,Photogramm
3,3355684,1920.0,NaN,08/17/2017 12:00:00 AM +0000,Constructed,MULTIPOLYGON (((-73.94029215265738 40.64108287...,858061,11.200000,5110.0,32.0,248.678169,63.940817,3.049720e+09,3.049720e+09,Photogramm
4,3131737,1915.0,NaN,08/22/2017 12:00:00 AM +0000,Constructed,MULTIPOLYGON (((-73.98998983552244 40.62383804...,568078,24.980000,2100.0,44.0,1163.227669,165.608763,3.055100e+09,3.055100e+09,Photogramm


In [30]:
ages = ages[['BIN', 'CNSTRCT_YR']]
columns = ['bin', 'yearBuilt']
ages.columns = columns

In [31]:
ages['bin']= ages['bin'].astype(int)

In [32]:
ages["geoid"] = "0"
ages['county'] = '0'

In [64]:
# example request: 
# https://api.cityofnewyork.us/geoclient//v1/bin.json?bin=1079043&app_id=abc123&app_key=def456

def getLatLong(BIN, retryRun=0):  # returns CensusTract
    url = "https://api.cityofnewyork.us/geoclient//v1/bin.json?bin={0}&app_id={1}&app_key={2}".format(BIN, projectId, key)

    if retryRun == 15:  # Failed to get json data 3 times with this longitude and latitude so need to skip this one
        print("failed 15 times")
        return (str(0), str(0))
    try:
        print('BIN:',BIN)
        response = requests.get(url)
        dataJSON = response.json()
        data = dataJSON["bin"]
        ct = (data["latitudeInternalLabel"], data['longitudeInternalLabel'])
        return ct
    except:
        retryRun += 1
        print("******** Error on bin: " + str(BIN) + " ------ retrying " + str(retryRun))
        return getLatLong(BIN, retryRun)  # *****need to return the recursive function
    
def getCensusTract(longitude, latitude, retryRun = 0):
    url2 = "https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x={0}&y={1}&benchmark=Public_AR_Census2010&vintage=Census2010_Census2010&format=json".format(
        longitude, latitude)
    if retryRun == 15:  # Failed to get json data 11 times with this longitude and latitude so need to skip this one
        print("failed 15 times")
        return [str(0), str(0)]
    try:
        response = requests.get(url2)
        dataJSON = response.json()
        data = dataJSON["result"]
        county = data["geographies"]["Counties"][0]["BASENAME"]
        geoid = data["geographies"]["Census Tracts"][0]["GEOID"]
        #county = data["geographies"]["Counties"][0]["NAME"]
        return [str(county), str(geoid)]
    except:
        retryRun += 1
        print("******** Error on longitude, latitude: " + str(longitude) + "," + str(
            latitude) + " ------ retrying " + str(retryRun))
        return getCensusTract(longitude, latitude, retryRun)  # *****need to return the recursive function




In [49]:
for row in range(400, len(ages)):
    # b) using the BIN of each entry to find the census data and adding to the respective arrays to add to csv col later
    lat, long = getLatLong(int(ages.loc[row]["bin"].item()))
                                  # data.iat[i,4] = lat   # data.iat[i,5] = lon
    ct = getCensusTract(float(long), float(lat))
    ages.at[row, "geoid"] = ct[1]
    ages.at[row, "county"] = ct[0]
    print(ct)

BIN: 2017770
['Bronx', '36005042100']
BIN: 5093985
******** Error on longitude, latitude: -74.06654102353993,40.593581016062934 ------ retrying 1
['Richmond', '36085007400']
BIN: 4047447
['Queens', '36081041300']
BIN: 3072692
['Kings', '36047039300']
BIN: 5038242
******** Error on longitude, latitude: -74.15549980324582,40.59036050422304 ------ retrying 1
******** Error on longitude, latitude: -74.15549980324582,40.59036050422304 ------ retrying 2
******** Error on longitude, latitude: -74.15549980324582,40.59036050422304 ------ retrying 3
******** Error on longitude, latitude: -74.15549980324582,40.59036050422304 ------ retrying 4
******** Error on longitude, latitude: -74.15549980324582,40.59036050422304 ------ retrying 5
******** Error on longitude, latitude: -74.15549980324582,40.59036050422304 ------ retrying 6
******** Error on longitude, latitude: -74.15549980324582,40.59036050422304 ------ retrying 7
******** Error on longitude, latitude: -74.15549980324582,40.59036050422304 --

******** Error on longitude, latitude: -73.8335527913874,40.68641011090419 ------ retrying 3
['Queens', '36081011000']
BIN: 5035729
******** Error on longitude, latitude: -74.1647062412352,40.59760493794361 ------ retrying 1
******** Error on longitude, latitude: -74.1647062412352,40.59760493794361 ------ retrying 2
******** Error on longitude, latitude: -74.1647062412352,40.59760493794361 ------ retrying 3
******** Error on longitude, latitude: -74.1647062412352,40.59760493794361 ------ retrying 4
******** Error on longitude, latitude: -74.1647062412352,40.59760493794361 ------ retrying 5
******** Error on longitude, latitude: -74.1647062412352,40.59760493794361 ------ retrying 6
******** Error on longitude, latitude: -74.1647062412352,40.59760493794361 ------ retrying 7
['Richmond', '36085029103']
BIN: 5104842
******** Error on longitude, latitude: -74.07601278474849,40.595254958537936 ------ retrying 1
******** Error on longitude, latitude: -74.07601278474849,40.595254958537936 ----

******** Error on longitude, latitude: -73.8076757839876,40.69596825348192 ------ retrying 10
******** Error on longitude, latitude: -73.8076757839876,40.69596825348192 ------ retrying 11
******** Error on longitude, latitude: -73.8076757839876,40.69596825348192 ------ retrying 12
******** Error on longitude, latitude: -73.8076757839876,40.69596825348192 ------ retrying 13
['Queens', '36081020800']
BIN: 3416235
******** Error on longitude, latitude: -73.99776305649603,40.62062826031262 ------ retrying 1
******** Error on longitude, latitude: -73.99776305649603,40.62062826031262 ------ retrying 2
******** Error on longitude, latitude: -73.99776305649603,40.62062826031262 ------ retrying 3
******** Error on longitude, latitude: -73.99776305649603,40.62062826031262 ------ retrying 4
['Kings', '36047026200']
BIN: 5024941
******** Error on longitude, latitude: -74.14072015349586,40.62479384205251 ------ retrying 1
******** Error on longitude, latitude: -74.14072015349586,40.62479384205251 -

******** Error on longitude, latitude: -73.81291298728708,40.78559321037996 ------ retrying 6
******** Error on longitude, latitude: -73.81291298728708,40.78559321037996 ------ retrying 7
******** Error on longitude, latitude: -73.81291298728708,40.78559321037996 ------ retrying 8
******** Error on longitude, latitude: -73.81291298728708,40.78559321037996 ------ retrying 9
******** Error on longitude, latitude: -73.81291298728708,40.78559321037996 ------ retrying 10
******** Error on longitude, latitude: -73.81291298728708,40.78559321037996 ------ retrying 11
******** Error on longitude, latitude: -73.81291298728708,40.78559321037996 ------ retrying 12
******** Error on longitude, latitude: -73.81291298728708,40.78559321037996 ------ retrying 13
******** Error on longitude, latitude: -73.81291298728708,40.78559321037996 ------ retrying 14
******** Error on longitude, latitude: -73.81291298728708,40.78559321037996 ------ retrying 15
failed 15 times
['0', '0']
BIN: 5084777
******** Error

******** Error on longitude, latitude: -73.91928988635486,40.70406321768269 ------ retrying 6
******** Error on longitude, latitude: -73.91928988635486,40.70406321768269 ------ retrying 7
******** Error on longitude, latitude: -73.91928988635486,40.70406321768269 ------ retrying 8
['Kings', '36047044500']
BIN: 3136660
['Kings', '36047022200']
BIN: 5060243
******** Error on longitude, latitude: -74.13677233588895,40.57049545855072 ------ retrying 1
******** Error on longitude, latitude: -74.13677233588895,40.57049545855072 ------ retrying 2
['Richmond', '36085013800']
BIN: 5102331
['Richmond', '36085013400']
BIN: 3093109
******** Error on longitude, latitude: -73.87763396812772,40.67788251678356 ------ retrying 1
******** Error on longitude, latitude: -73.87763396812772,40.67788251678356 ------ retrying 2
******** Error on longitude, latitude: -73.87763396812772,40.67788251678356 ------ retrying 3
['Kings', '36047119200']
BIN: 4524940
******** Error on longitude, latitude: -73.864288613

...

['Queens', '36081054700']
BIN: 3084536
['Kings', '36047115800']
BIN: 4183070
******** Error on longitude, latitude: -73.86537769424426,40.691373121742956 ------ retrying 1
******** Error on longitude, latitude: -73.86537769424426,40.691373121742956 ------ retrying 2
['Queens', '36081000400']
BIN: 1049965
******** Error on longitude, latitude: -73.95095398558199,40.77607590457652 ------ retrying 1
******** Error on longitude, latitude: -73.95095398558199,40.77607590457652 ------ retrying 2
******** Error on longitude, latitude: -73.95095398558199,40.77607590457652 ------ retrying 3
******** Error on longitude, latitude: -73.95095398558199,40.77607590457652 ------ retrying 4
['New York', '36061014601']
BIN: 3111554
******** Error on longitude, latitude: -73.94317903578525,40.64229532846881 ------ retrying 1
['Kings', '36047083200']
BIN: 4245236
['Queens', '36081005800']
BIN: 4434499
['Queens', '36081055900']
BIN: 5049328
******** Error on longitude, latitude: -74.0916383032271,40.5885956

******** Error on longitude, latitude: -73.95965655836422,40.720635360215205 ------ retrying 7
******** Error on longitude, latitude: -73.95965655836422,40.720635360215205 ------ retrying 8
******** Error on longitude, latitude: -73.95965655836422,40.720635360215205 ------ retrying 9
['Kings', '36047055700']
BIN: 3010604
******** Error on longitude, latitude: -74.00531917942729,40.65057104312779 ------ retrying 1
******** Error on longitude, latitude: -74.00531917942729,40.65057104312779 ------ retrying 2
******** Error on longitude, latitude: -74.00531917942729,40.65057104312779 ------ retrying 3
******** Error on longitude, latitude: -74.00531917942729,40.65057104312779 ------ retrying 4
******** Error on longitude, latitude: -74.00531917942729,40.65057104312779 ------ retrying 5
******** Error on longitude, latitude: -74.00531917942729,40.65057104312779 ------ retrying 6
******** Error on longitude, latitude: -74.00531917942729,40.65057104312779 ------ retrying 7
******** Error on l

******** Error on longitude, latitude: -73.74850706043098,40.71727988431148 ------ retrying 3
******** Error on longitude, latitude: -73.74850706043098,40.71727988431148 ------ retrying 4
******** Error on longitude, latitude: -73.74850706043098,40.71727988431148 ------ retrying 5
******** Error on longitude, latitude: -73.74850706043098,40.71727988431148 ------ retrying 6
******** Error on longitude, latitude: -73.74850706043098,40.71727988431148 ------ retrying 7
['Queens', '36081054200']
BIN: 4475941
******** Error on longitude, latitude: -73.88260049149906,40.71356421518037 ------ retrying 1
******** Error on longitude, latitude: -73.88260049149906,40.71356421518037 ------ retrying 2
******** Error on longitude, latitude: -73.88260049149906,40.71356421518037 ------ retrying 3
******** Error on longitude, latitude: -73.88260049149906,40.71356421518037 ------ retrying 4
******** Error on longitude, latitude: -73.88260049149906,40.71356421518037 ------ retrying 5
******** Error on lon

******** Error on longitude, latitude: -73.77033467869975,40.69446976689698 ------ retrying 2
******** Error on longitude, latitude: -73.77033467869975,40.69446976689698 ------ retrying 3
******** Error on longitude, latitude: -73.77033467869975,40.69446976689698 ------ retrying 4
******** Error on longitude, latitude: -73.77033467869975,40.69446976689698 ------ retrying 5
******** Error on longitude, latitude: -73.77033467869975,40.69446976689698 ------ retrying 6
******** Error on longitude, latitude: -73.77033467869975,40.69446976689698 ------ retrying 7
['Queens', '36081042400']
BIN: 3161582
['Kings', '36047027200']
BIN: 3015582
******** Error on longitude, latitude: -74.02018651218924,40.64441009987551 ------ retrying 1
******** Error on longitude, latitude: -74.02018651218924,40.64441009987551 ------ retrying 2
******** Error on longitude, latitude: -74.02018651218924,40.64441009987551 ------ retrying 3
******** Error on longitude, latitude: -74.02018651218924,40.64441009987551 -

ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
count = 0
for row in range(1650, len(ages)):
    # b) using the BIN of each entry to find the census data and adding to the respective arrays to add to csv col later
    lat, long = getLatLong(int(ages.loc[row]["bin"].item()))
                                  # data.iat[i,4] = lat   # data.iat[i,5] = lon
    ct = getCensusTract(float(long), float(lat))
    ages.at[row, "geoid"] = ct[1]
    ages.at[row, "county"] = ct[0]
    count += 1
    print(ct, count)

BIN: 3229372
['Kings', '36047096800'] 1
BIN: 3217687
['Kings', '36047067600'] 2
BIN: 3349344
['Kings', '36047097400'] 3
BIN: 5045752
['Richmond', '36085007400'] 4
BIN: 4588788
['Queens', '36081020800'] 5
BIN: 5048345
['Richmond', '36085009601'] 6
BIN: 4500912
['Queens', '36081042400'] 7
BIN: 3161582
['Kings', '36047027200'] 8
BIN: 3015582
******** Error on longitude, latitude: -74.02018651218924,40.64441009987551 ------ retrying 1
['Kings', '36047002200'] 9
BIN: 4540348
['Queens', '36081091601'] 10
BIN: 1089283
******** Error on bin: 1089283 ------ retrying 1
BIN: 1089283
******** Error on bin: 1089283 ------ retrying 2
BIN: 1089283
******** Error on bin: 1089283 ------ retrying 3
BIN: 1089283
******** Error on bin: 1089283 ------ retrying 4
BIN: 1089283
******** Error on bin: 1089283 ------ retrying 5
BIN: 1089283
******** Error on bin: 1089283 ------ retrying 6
BIN: 1089283
******** Error on bin: 1089283 ------ retrying 7
BIN: 1089283
******** Error on bin: 1089283 ------ retrying 8


******** Error on longitude, latitude: -73.79591751872861,40.68598995322466 ------ retrying 6
******** Error on longitude, latitude: -73.79591751872861,40.68598995322466 ------ retrying 7
******** Error on longitude, latitude: -73.79591751872861,40.68598995322466 ------ retrying 8
******** Error on longitude, latitude: -73.79591751872861,40.68598995322466 ------ retrying 9
['Queens', '36081019000'] 31
BIN: 4457488
['Queens', '36081025400'] 32
BIN: 3027115
['Kings', '36047014900'] 33
BIN: 4497269
['Queens', '36081014800'] 34
BIN: 4572780
******** Error on longitude, latitude: -73.82455508875567,40.68832083260177 ------ retrying 1
******** Error on longitude, latitude: -73.82455508875567,40.68832083260177 ------ retrying 2
******** Error on longitude, latitude: -73.82455508875567,40.68832083260177 ------ retrying 3
******** Error on longitude, latitude: -73.82455508875567,40.68832083260177 ------ retrying 4
******** Error on longitude, latitude: -73.82455508875567,40.68832083260177 -----

******** Error on longitude, latitude: -73.857704426339,40.84669163049757 ------ retrying 7
['Bronx', '36005024400'] 74
BIN: 2073098
['Bronx', '36005018400'] 75
BIN: 4025693
['Queens', '36081030903'] 76
BIN: 5028965
['Richmond', '36085025100'] 77
BIN: 4510813
['Queens', '36081036600'] 78
BIN: 2014345
['Bronx', '36005025300'] 79
BIN: 2083951
['Bronx', '36005028500'] 80
BIN: 2111187
['Bronx', '36005034200'] 81
BIN: 5016053
['Richmond', '36085004700'] 82
BIN: 4556052
['Queens', '36081009800'] 83
BIN: 2050531
['Bronx', '36005031600'] 84
BIN: 4295811
['Queens', '36081088400'] 85
BIN: 3132397
******** Error on longitude, latitude: -73.9917759473596,40.623172409441125 ------ retrying 1
******** Error on longitude, latitude: -73.9917759473596,40.623172409441125 ------ retrying 2
['Kings', '36047024800'] 86
BIN: 2100454
['Bronx', '36005045600'] 87
BIN: 3025527
['Kings', '36047015500'] 88
BIN: 3195209
['Kings', '36047037401'] 89
BIN: 3102841
['Kings', '36047086000'] 90
BIN: 2001059
******** Erro

******** Error on longitude, latitude: -73.73904105742388,40.718246934808946 ------ retrying 2
******** Error on longitude, latitude: -73.73904105742388,40.718246934808946 ------ retrying 3
******** Error on longitude, latitude: -73.73904105742388,40.718246934808946 ------ retrying 4
******** Error on longitude, latitude: -73.73904105742388,40.718246934808946 ------ retrying 5
['Queens', '36081056600'] 128
BIN: 3046680
******** Error on longitude, latitude: -73.92382083270341,40.684282007811746 ------ retrying 1
******** Error on longitude, latitude: -73.92382083270341,40.684282007811746 ------ retrying 2
******** Error on longitude, latitude: -73.92382083270341,40.684282007811746 ------ retrying 3
******** Error on longitude, latitude: -73.92382083270341,40.684282007811746 ------ retrying 4
******** Error on longitude, latitude: -73.92382083270341,40.684282007811746 ------ retrying 5
******** Error on longitude, latitude: -73.92382083270341,40.684282007811746 ------ retrying 6
*******

******** Error on longitude, latitude: -73.93608095613064,40.61921615749142 ------ retrying 9
******** Error on longitude, latitude: -73.93608095613064,40.61921615749142 ------ retrying 10
******** Error on longitude, latitude: -73.93608095613064,40.61921615749142 ------ retrying 11
******** Error on longitude, latitude: -73.93608095613064,40.61921615749142 ------ retrying 12
******** Error on longitude, latitude: -73.93608095613064,40.61921615749142 ------ retrying 13
******** Error on longitude, latitude: -73.93608095613064,40.61921615749142 ------ retrying 14
******** Error on longitude, latitude: -73.93608095613064,40.61921615749142 ------ retrying 15
failed 15 times
['0', '0'] 162
BIN: 2017760
['Bronx', '36005041900'] 163
BIN: 5024325
******** Error on longitude, latitude: -74.13558907489605,40.633034385654355 ------ retrying 1
******** Error on longitude, latitude: -74.13558907489605,40.633034385654355 ------ retrying 2
******** Error on longitude, latitude: -74.13558907489605,40

...

['Bronx', '36005042600'] 4608
BIN: 2013109
['Bronx', '36005036300'] 4609
BIN: 2018186
******** Error on longitude, latitude: -73.87538645433553,40.87738137675728 ------ retrying 1
['Bronx', '36005042300'] 4610
BIN: 5014723
['Richmond', '36085001700'] 4611
BIN: 5024444
['Richmond', '36085021300'] 4612
BIN: 5045578
['Richmond', '36085002002'] 4613
BIN: 5109111
['Richmond', '36085031902'] 4614
BIN: 5014331
******** Error on longitude, latitude: -74.08307306865419,40.62059826652904 ------ retrying 1
['Richmond', '36085002900'] 4615
BIN: 5005128
['Richmond', '36085013302'] 4616
BIN: 5007591
['Richmond', '36085012100'] 4617
BIN: 5029127
['Richmond', '36085025100'] 4618
BIN: 5012492
******** Error on longitude, latitude: -74.13774973566426,40.61332968395493 ------ retrying 1
******** Error on longitude, latitude: -74.13774973566426,40.61332968395493 ------ retrying 2
['Richmond', '36085018901'] 4619
BIN: 5005966
['Richmond', '36085012500'] 4620
BIN: 5002632
******** Error on longitude, latitu

******** Error on longitude, latitude: -73.85168435820614,40.73414718502566 ------ retrying 1
['Queens', '36081074500'] 4722
BIN: 4078137
['Queens', '36081073100'] 4723
BIN: 4052742
['Queens', '36081075701'] 4724
BIN: 4062809
['Queens', '36081049302'] 4725
BIN: 4026310
['Queens', '36081030902'] 4726
BIN: 4044890
['Queens', '36081039900'] 4727
BIN: 4038776
['Queens', '36081047300'] 4728
BIN: 4541838
['Queens', '36081000100'] 4729
BIN: 4069600
['Queens', '36081067900'] 4730
BIN: 4001129
['Queens', '36081016900'] 4731
BIN: 4053605
******** Error on longitude, latitude: -73.90522429431034,40.73927325679069 ------ retrying 1
['Queens', '36081024500'] 4732
BIN: 4433603
['Queens', '36081059900'] 4733
BIN: 4470602
['Queens', '36081035100'] 4734
BIN: 4058592
['Queens', '36081049302'] 4735
BIN: 4012795
******** Error on longitude, latitude: -73.90788009620431,40.76545482045824 ------ retrying 1
['Queens', '36081014500'] 4736
BIN: 4005271
['Queens', '36081001900'] 4737
BIN: 4054223
['Queens', '36

['Queens', '36081010100'] 4845
BIN: 1056680
******** Error on longitude, latitude: -73.96517683043083,40.80216426710208 ------ retrying 1
******** Error on longitude, latitude: -73.96517683043083,40.80216426710208 ------ retrying 2
['New York', '36061019500'] 4846
BIN: 1013096
******** Error on longitude, latitude: -74.00252615953471,40.74336347949681 ------ retrying 1
['New York', '36061008900'] 4847
BIN: 1003384
['New York', '36061002700'] 4848
BIN: 1025437
['New York', '36061013900'] 4849
BIN: 1055989
['New York', '36061019300'] 4850
BIN: 2003546
['Bronx', '36005020100'] 4851
BIN: 1010258
******** Error on longitude, latitude: -74.00173556628141,40.73314478152258 ------ retrying 1
['New York', '36061007100'] 4852
BIN: 1038524
******** Error on longitude, latitude: -73.96835532559227,40.75669040015129 ------ retrying 1
******** Error on longitude, latitude: -73.96835532559227,40.75669040015129 ------ retrying 2
['New York', '36061009800'] 4853
BIN: 3217514
['Kings', '36047073000'] 48

['Bronx', '36005036000'] 4934
BIN: 2058576
['Bronx', '36005039600'] 4935
BIN: 2070300
['Bronx', '36005044400'] 4936
BIN: 2050814
******** Error on longitude, latitude: -73.84417546849302,40.860672152526554 ------ retrying 1
['Bronx', '36005031400'] 4937
BIN: 2066111
['Bronx', '36005042600'] 4938
BIN: 2042330
['Bronx', '36005020000'] 4939
BIN: 2045328
['Bronx', '36005024400'] 4940
BIN: 2051624
['Bronx', '36005032400'] 4941
BIN: 2074958
['Bronx', '36005026400'] 4942
BIN: 4059594
['Queens', '36081059500'] 4943
BIN: 4439303
['Queens', '36081052100'] 4944
BIN: 4024857
******** Error on longitude, latitude: -73.89305930772363,40.759796077066426 ------ retrying 1
['Queens', '36081030903'] 4945
BIN: 4077063
['Queens', '36081072300'] 4946
BIN: 4068314
['Queens', '36081066300'] 4947
BIN: 4051897
['Queens', '36081074100'] 4948
BIN: 4051586
['Queens', '36081074300'] 4949
BIN: 4036474
['Queens', '36081027500'] 4950
BIN: 4609277
******** Error on longitude, latitude: -73.87147319174628,40.7093665629

['Bronx', '36005028100'] 5057
BIN: 2069704
['Bronx', '36005041800'] 5058
BIN: 2105743
['Bronx', '36005026602'] 5059
BIN: 2016957
['Bronx', '36005041500'] 5060
BIN: 2063510


In [ ]:
ages.to_csv(r'../data/processed/building_temp.csv', encoding='utf-8',index=False)